In [1]:
import numpy as np
import pandas as pd
import os
import json
import pickle
import random
import math
import re

In [2]:
def uniqueid():
    seed = random.getrandbits(64)
    while True:
        yield seed
        seed += 1

In [3]:
root_dir = "."
input_path = os.path.join(root_dir, 'raw_data', 'all_data_with_article_title_body.csv')
output_fpath = os.path.join(root_dir, 'processed', 'dictionary.pickle')

In [4]:
f = open('processed/duplicates_url_to_label.json')
dups_url_to_label = json.load(f)

In [5]:
dictionary = []
url_to_label = {}

In [6]:
df = pd.read_csv(input_path)  # Load all the concatenated train, val, and test data

In [7]:
columns = ['url', 'title', 'body']
df[columns].head()

,url,title,body
0,http://www.politifact.com/truth-o-meter/statem...,"PolitiFact | Prior to Benghazi, were there 13 ...",As the U.S. House of Representatives was ready...
1,http://www.politifact.com/truth-o-meter/statem...,PolitiFact | Barack Obama said he&#39;s cut ta...,\nIn his speech at the Democratic National Com...
2,http://www.politifact.com/punditfact/statement...,PolitiFact | Juan Williams: No. 1 cause of dea...,"Amid ongoing protests in Ferguson, Mo., the na..."
3,http://www.snopes.com/donald-trumps-son-game-h...,Are These Donald Trump&#039;s Sons in Game-Hun...,NaN
4,http://www.politifact.com/truth-o-meter/statem...,PolitiFact | Hillary Clinton blames high-up Ru...,Editor's note: We've attached an update to thi...


In [8]:
df_final = df[columns]

In [9]:
df_final.shape

(55680, 3)

In [11]:
df_final['body'].iloc[0]

'As the U.S. House of Representatives was readying a new special committee to investigate the terrorist attack in Benghazi, Libya, many Democrats were arguing that continuing to probe the Sept. 11, 2012, attack -- which killed four Americans, including U.S. Ambassador Chris Stevens -- amounted to a political witch hunt.\nOn May 5, 2014, Rep. John Garamendi, D-Calif., told MSNBC host Ed Schultz that there has already been exhaustive testimony and investigation of the incident.\n"This thing is just going on and on to boredom actually," Garamendi said. "The Armed Services Committee actually did a hearing and the result was there’s nothing here. That’s obviously a great tragedy, but Ed, during the George W. Bush period, there were 13 attacks on various embassies and consulates around the world. Sixty people died. In Karachi, there was a death of one of our diplomats, and those were not investigated during that period of time because it was a tragedy."\nReaders asked us whether it’s true th

In [12]:
#re.sub('[^A-Za-z0-9]+', '', mystring), use this if isalnum() doesn't work
def clean_up(s: str):
    if type(s) is float and math.isnan(s):
        return ""
    else:
        s = re.sub("\n"," ",s)
        s = ''.join(e for e in s if e.isalnum() or e is ' ')
        s = re.sub("\s+"," ",s)
        return s.strip()

In [13]:
label_id_set = set()
label_id_record = {}
label_count = 272  # 0 - 271 are reserved for duplicate Titles and Description for URLs
#unique_sequence = uniqueid()
for idx in range(df_final.shape[0]):
    row = df_final.iloc[idx]
    record = {}
    label = label_count
    
    if row['url'].strip() in url_to_label:
        continue
    
    if row['url'].strip() in dups_url_to_label:
        label = dups_url_to_label[row['url'].strip()]
        if label in label_id_set:
            url_to_label[row['url']] = str(label)
            continue
    
    #label_id = str(next(unique_sequence))
    url_to_label[row['url']] = str(label)
    record['cui'] = str(label)
    
    
    record['title'] = clean_up(row['title'])
    record['description'] = clean_up(row['body'])
    record['type'] = ""
    label_id_set.add(label)
    dictionary.append(record)
    if label == label_count:
        label_count+=1

In [13]:
assert len(dictionary) == len(label_id_set)

In [14]:
len(dictionary)

3540

In [15]:
df = pd.DataFrame(dictionary)

In [16]:
root = "."
input_path = os.path.join(root, 'processed', 'dictionary_cleaned.pickle')
file = open(input_path,'rb')
dic_cleaned = pickle.load(file)

In [17]:
df_clean = pd.DataFrame(dic_cleaned)

In [18]:
summary_list = []
for i in range(df.shape[0]):
    summary = df_clean[df_clean['title'] == df['title'].iloc[i]]['summary'].values[0]
    summary_list.append(summary)

In [19]:
df['summary'] = summary_list

In [20]:
df.columns

Index(['cui', 'title', 'description', 'type', 'summary'], dtype='object')

In [21]:
df.head()

,cui,title,description,type,summary
0,272,PolitiFact Prior to Benghazi were there 13 att...,As the US House of Representatives was readyin...,,Rep John Garamendi said there were 13 attacks ...
1,273,PolitiFact Barack Obama said he39s cut taxes f...,In his speech at the Democratic National Commi...,,Obama has signed off on two major tax cuts for...
2,274,PolitiFact Juan Williams No 1 cause of death f...,Amid ongoing protests in Ferguson Mo the natio...,,Civil rights activists and black leaders have ...
3,14,Are These Donald Trump039s Sons in GameHunting...,,,
4,275,PolitiFact Hillary Clinton blames highup Russi...,Editors note Weve attached an update to this f...,,Clinton We have 17 intelligence agencies civil...


In [22]:
df_clean.head()

,cui,title,description,type,summary
0,10930383388364076528,PolitiFact Prior to Benghazi were there 13 att...,As the US House of Representatives was readyin...,,Rep John Garamendi said there were 13 attacks ...
1,10930383388364076529,PolitiFact Barack Obama said he39s cut taxes f...,In his speech at the Democratic National Commi...,,Obama has signed off on two major tax cuts for...
2,10930383388364076530,PolitiFact Juan Williams No 1 cause of death f...,Amid ongoing protests in Ferguson Mo the natio...,,Civil rights activists and black leaders have ...
3,10930383388364076531,Are These Donald Trump039s Sons in GameHunting...,,,
4,10930383388364076532,PolitiFact Hillary Clinton blames highup Russi...,Editors note Weve attached an update to this f...,,Clinton We have 17 intelligence agencies civil...


In [23]:
dictionary_final = []
for i in range(df.shape[0]):
    record = {}
    record['cui'] = df['cui'].iloc[i]
    record['title'] = df['title'].iloc[i]
    record['description'] = df['description'].iloc[i]
    record['summary'] = df['summary'].iloc[i]
    dictionary_final.append(record)

In [24]:
with open(output_fpath, 'wb') as write_handle:
    pickle.dump(dictionary_final, write_handle, protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
with open("processed/url_cui_mapping.json", "w") as outfile:
    json.dump(url_to_label, outfile)

In [14]:
dictionary

[{'cui': '272',
  'title': 'PolitiFact Prior to Benghazi were there 13 attacks on embassies and 60 deaths under President George W Bush',
  'description': 'As the US House of Representatives was readying a new special committee to investigate the terrorist attack in Benghazi Libya many Democrats were arguing that continuing to probe the Sept 11 2012 attack which killed four Americans including US Ambassador Chris Stevens amounted to a political witch hunt On May 5 2014 Rep John Garamendi DCalif told MSNBC host Ed Schultz that there has already been exhaustive testimony and investigation of the incident This thing is just going on and on to boredom actually Garamendi said The Armed Services Committee actually did a hearing and the result was theres nothing here Thats obviously a great tragedy but Ed during the George W Bush period there were 13 attacks on various embassies and consulates around the world Sixty people died In Karachi there was a death of one of our diplomats and those we

In [27]:
dups_url_to_label

{'http://www.snopes.com/photos/signs/islamicagenda.asp': 0,
 'https://www.snopes.com/photos/signs/islamicagenda.asp': 0,
 'https://www.snopes.com/history/document/communistrules.asp': 1,
 'https://www.snopes.com/fact-check/communist-rules-for-revolution/': 1,
 'http://www.snopes.com/business/consumer/bastard.asp': 2,
 'https://www.snopes.com/business/consumer/bastard.asp': 2,
 'http://www.snopes.com/donald-trump-haters-and-losers/': 3,
 'https://www.snopes.com/fact-check/donald-trump-haters-and-losers/': 3,
 'http://www.snopes.com/photos/politics/fergusonsign.asp': 4,
 'https://www.snopes.com/photos/politics/fergusonsign.asp': 4,
 'http://www.snopes.com/rivers-of-blood-in-dhaka-bangladesh/': 5,
 'https://www.snopes.com/fact-check/rivers-of-blood-in-dhaka-bangladesh/': 5,
 'https://www.snopes.com/fact-check/unthemely-behavior/': 6,
 'http://www.snopes.com/radiotv/tv/trek.asp': 6,
 'https://www.snopes.com/language/document/hiringwomen.asp': 7,
 'https://www.snopes.com/fact-check/crass-tr

In [28]:
url_to_label['http://www.snopes.com/photos/signs/islamicagenda.asp']

'0'

In [29]:
url_to_label['https://www.snopes.com/photos/signs/islamicagenda.asp']

'0'